<a href="https://colab.research.google.com/github/greeneley/aivivn/blob/master/ALVIVN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

**IMPORT FILE DATAFRAME**

In [51]:

def import_file():
  from google.colab import files
  uploaded = files.upload()
  import io
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
  return df
df = import_file()

Saving train.csv to train (4).csv
User uploaded file "train.csv" with length 1675800 bytes


**DATA PREPROCESSING**

In [0]:
def data_preprocessing(data):
  data_process = data.copy()
  data_process = data_process.apply(lambda y:  y.replace(y['ZONE_CODE'], 1) if (y['ZONE_CODE'] == 'ZONE01') else ( y.replace(y['ZONE_CODE'], 2) if (y['ZONE_CODE'] == 'ZONE02') else y.replace(y['ZONE_CODE'], 3)) , axis=1)
  data_process['UPDATE_TIME'] = data_process['UPDATE_TIME'].str.replace('-',"")
  return data_process
df_process = data_preprocessing(df)

**MODEL TRAINING**

In [63]:
X_train = df_process.iloc[:,0:3]
Y_train = df_process.iloc[:,-2:-1]
# =================================
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(df_process.iloc[:,0:3], df_process.iloc[:,3:5])
# =================================
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor

knn = KNeighborsRegressor()
regr = MultiOutputRegressor(knn)
regr.fit(X_train, Y_train)





MultiOutputRegressor(estimator=KNeighborsRegressor(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=5,
                                                   p=2, weights='uniform'),
                     n_jobs=None)

**MODEL TESTING**

In [55]:
test_data = import_file()


Saving test_id.csv to test_id (3).csv
User uploaded file "test_id.csv" with length 55871 bytes


,id,UPDATE_TIME,ZONE_CODE,HOUR_ID
0,0,2019-03-10,ZONE01,0
1,1,2019-03-10,ZONE01,1
2,2,2019-03-10,ZONE01,2
3,3,2019-03-10,ZONE01,3
4,4,2019-03-10,ZONE01,4
5,5,2019-03-10,ZONE01,5
6,6,2019-03-10,ZONE01,6
7,7,2019-03-10,ZONE01,7
8,8,2019-03-10,ZONE01,8
9,9,2019-03-10,ZONE01,9


In [0]:
test_data = data_preprocessing(test_data)

In [0]:
test_data = test_data.drop(['id'], axis=1)

In [59]:
test_data.shape

(2226, 3)

In [0]:
predict = regr.predict(test_data)

In [0]:
predict = pd.DataFrame(predict)
predict.loc[:,0] = round(predict.loc[:,0], 2)
predict.loc[:,1] = round(predict.loc[:,0])
predict.head()
predict.to_csv('./result.csv')

In [0]:
pd.DataFrame(predict).to_csv("./result.csv")

In [0]:
from google.colab import files
files.download('result.csv')
